In [1]:
from ngsolve import *
import netgen.gui
from netgen.occ import *
import netgen.meshing as ngm
from mpi4py.MPI import COMM_WORLD

if COMM_WORLD.rank == 0:
    d = 0.01
    box = Box ( (-d/2,-d/2,0), (d/2,d/2,0.1) ) + Box( (-d/2, -3*d/2,0.1), (d/2, 3*d/2, 0.1+d) )
    box.faces.Min(Z).name = "bottom"
    box.faces.Max(Z).name = "top"
    mesh = Mesh(OCCGeometry(box).GenerateMesh(maxh=0.05).Distribute(COMM_WORLD))
else:
    mesh = Mesh(ngm.Mesh.Receive(COMM_WORLD))

E, nu = 210, 0.2
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

def C(u):
    F = Id(u.dim) + Grad(u)
    return F.trans * F

def NeoHooke (C):
    # return 0.5*mu*InnerProduct(C-Id(3), C-Id(3))
    return 0.5*mu*(Trace(C-Id(3)) + 2*mu/lam*Det(C)**(-lam/2/mu)-1)

loadfactor = Parameter(1)
force = loadfactor * CF ( (-y, x, 0) )

fes = H1(mesh, order=3, dirichlet="bottom", dim=mesh.dim)
u,v = fes.TnT()

a = BilinearForm(fes)
a += Variation(NeoHooke(C(u)).Compile()*dx)
a += ((Id(3)+Grad(u.Trace()))*force)*v*ds("top")

optfile ./ng.opt does not exist - using default values
togl-version : 2
OCC module loaded
loading ngsolve library
NGSolve-6.2.2404-51-gaa4ae2b4e
Using Lapack
Including sparse direct solver Pardiso
Running parallel using 20 thread(s)
 Face 1 / 10 (parameter space projection)
 Face 2 / 10 (parameter space projection)
 Face 3 / 10 (parameter space projection)
 Face 4 / 10 (parameter space projection)
 Face 5 / 10 (parameter space projection)
 Face 6 / 10 (parameter space projection)
 Face 7 / 10 (parameter space projection)
 Face 8 / 10 (parameter space projection)
 Face 9 / 10 (parameter space projection)
 Face 10 / 10 (parameter space projection)
 Delaunay meshing
 Volume Optimization


In [4]:
from ngsPETSc import NonLinearSolver
gfu_petsc = GridFunction(fes)
gfu_ngs = GridFunction(fes)
gfu_ngs.vec[:] = 0; gfu_petsc.vec[:] = 0
gfu_history_ngs = GridFunction(fes, multidim=0)
gfu_history_petsc = GridFunction(fes, multidim=0)
numsteps = 30
for step in range(numsteps):
    print("step", step)
    loadfactor.Set(720*step/numsteps)
    solver = NonLinearSolver(fes, a=a, objective=False,
                             solverParameters={"snes_type": "newtonls",
                                               "snes_max_it": 200,
                                               "snes_monitor": "",
                                               "ksp_type": "preonly",
                                               "pc_type": "lu",
                                               "snes_linesearch_type": "basic",
                                               "snes_linesearch_damping": 1.0,
                                               "snes_linesearch_max_it": 100})
    gfu_petsc = solver.solve(gfu_petsc)
    solvers.Newton (a, gfu_ngs, printing=True, dampfactor=0.5)
    gfu_history_ngs.AddMultiDimComponent(gfu_ngs.vec)
    gfu_history_petsc.AddMultiDimComponent(gfu_petsc.vec)

/home/sebastian-ferra/tesis/lib/python3.12/site-packages/ngsPETSc/eps.py:10: UserWarning: Import Warning: it was not possible to import SLEPc
  warnings.warn("Import Warning: it was not possible to import SLEPc")


step 0
  0 SNES Function norm 0.000000000000e+00
Newton iteration  0
err =  0.0
step 1
  0 SNES Function norm 2.043515121443e-05
  1 SNES Function norm 9.513438371029e-03
  2 SNES Function norm 1.057101410127e-03
  3 SNES Function norm 1.986956360024e-04
  4 SNES Function norm 4.372422086116e-05
  5 SNES Function norm 4.743360901125e-05
  6 SNES Function norm 9.585281720350e-06
  7 SNES Function norm 5.123196667497e-06
  8 SNES Function norm 2.042222716179e-07
  9 SNES Function norm 2.829563346841e-09
 10 SNES Function norm 6.555704774520e-14
Newton iteration  0
err =  0.0005398283476964536
Newton iteration  1
err =  0.0015100392784207362
Newton iteration  2
err =  0.00028725946041557697
Newton iteration  3
err =  0.0021845348064314166
Newton iteration  4
err =  0.00033400650555226864
Newton iteration  5
err =  0.000950467615857466
Newton iteration  6
err =  0.0009605301181392383
Newton iteration  7
err =  0.025752507425303776
Newton iteration  8
err =  0.00923970655889128
Newton itera

In [3]:
cd NonLInearElasticity/

/home/sebastian-ferra/tesis/NonLInearElasticity


/home/sebastian-ferra/tesis/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
